# Template Explanation
I use this notebook multiple times across the life of project, both as a starting overview and a point of reference between various cleaning and transformation steps.

It uses a personalized version of the [cookiecutter data science](https://drivendata.github.io/cookiecutter-data-science/) file structure to simplify file selection.

In addition to basic built-in `pandas` overview tools, like `.info()` and `.describe()`, I added my own functions for common tasks including:
*  Creating a singular table of the same data from `.info()` and `.describe()`
    * Writing the table to `.xlsx` with frozen column headers and empty columns for note-taking.
        * `Notes` - General info that I need to remember, including questions
        * `Drop?` - Mark if the column can be dropped
        * `Change Dtype?` - Add what the column data type should be changed to
*  Displaying an overview of unique values by column

# Notebook Conclusions
Update this section with any findings and notes.

# Set Up

In [124]:
## Authorize Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Library imports

In [125]:
#general analysis
import pandas as pd
import pprint as pp
import re
import numpy as np
import seaborn as sns

#file management
from pathlib import Path
from datetime import datetime

#stop words counter
#from collections import Counter

In [126]:
#unique marker for new files
today = datetime.now().strftime("%Y-%m-%d")

## Display Preferences

In [127]:

#current preferences
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None) #change column display width
#pd.set_option('display.precision', 2) #displays 2 decimal places on all numbers
pd.set_option('display.float_format',  '{:.2f}'.format)
pd.set_option('display.memory_usage', 'deep')

# File Handling

In [128]:
# @title File Selection
# @markdown 1. Navigate to original file location in sidebar
# @markdown 2. right-click 'Copy path'
# @markdown 3. Paste into 'file_path' field

file_path = '/content/drive/MyDrive/data_analysis/dunnhumby/data/raw/transactions_200607.csv'  # @param {type: "string"}

# @markdown ---

In [129]:
#set up file_path
file_path = Path(file_path)

In [130]:
file_path

PosixPath('/content/drive/MyDrive/data_analysis/dunnhumby/data/raw/transactions_200607.csv')

In [150]:
#select main drive path
main_drive = file_path.parent.parent.parent
print(main_drive)

#create output filename string using today for unique id
output_filename = f'{file_path.stem}_{today}'
print(output_filename)

/content/drive/MyDrive/data_analysis/dunnhumby
transactions_200607_2024-03-15


### Read into pandas dataframe

In [132]:
keep_cols = []

df = pd.read_csv(file_path,
#                     usecols= keep_cols,
#                      sep='\t',
#                        nrows=100,
#                       engine='python',
#                     encoding='ISO-8859-1'
                        )

# DataFrame Overview

## Row and Column Count

In [133]:
df.shape

(250546, 22)

## `.info()`

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250546 entries, 0 to 250545
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SHOP_WEEK                 250546 non-null  int64  
 1   SHOP_DATE                 250546 non-null  int64  
 2   SHOP_WEEKDAY              250546 non-null  int64  
 3   SHOP_HOUR                 250546 non-null  int64  
 4   QUANTITY                  250546 non-null  int64  
 5   SPEND                     250546 non-null  float64
 6   PROD_CODE                 250546 non-null  object 
 7   PROD_CODE_10              250546 non-null  object 
 8   PROD_CODE_20              250546 non-null  object 
 9   PROD_CODE_30              250546 non-null  object 
 10  PROD_CODE_40              250546 non-null  object 
 11  CUST_CODE                 202211 non-null  object 
 12  CUST_PRICE_SENSITIVITY    202211 non-null  object 
 13  CUST_LIFESTAGE            175047 non-null  o

## Head and Tail Rows
First and last 5 rows

In [135]:
df.head(5)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200607,20060411,3,16,1,1.05,PRD0900011,CL00033,DEP00008,G00004,D00002,NaN,NaN,NaN,994100100100943,L,UM,Top Up,Fresh,STORE00001,LS,E02
1,200607,20060411,3,19,3,1.65,PRD0900035,CL00113,DEP00040,G00011,D00003,CUST0000173993,UM,OA,994100100257041,L,MM,Full Shop,Mixed,STORE00001,LS,E02
2,200607,20060414,6,21,1,0.71,PRD0900043,CL00148,DEP00052,G00015,D00003,NaN,NaN,NaN,994100100052871,L,LA,Top Up,Mixed,STORE00001,LS,E02
3,200607,20060414,6,14,1,0.46,PRD0900057,CL00107,DEP00037,G00010,D00003,CUST0000644893,LA,PE,994100100539059,L,LA,Full Shop,Fresh,STORE00001,LS,E02
4,200607,20060411,3,16,1,1.87,PRD0900058,CL00020,DEP00005,G00003,D00001,NaN,NaN,NaN,994100100100943,L,UM,Top Up,Fresh,STORE00001,LS,E02


In [136]:
df.tail(5)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
250541,200607,20060412,4,21,1,1.54,PRD0904358,CL00063,DEP00019,G00007,D00002,NaN,NaN,NaN,994100100007204,M,LA,Top Up,Mixed,STORE02908,SS,N02
250542,200607,20060411,3,21,1,1.54,PRD0904358,CL00063,DEP00019,G00007,D00002,NaN,NaN,NaN,994100100037346,L,UM,Top Up,Fresh,STORE02908,SS,N02
250543,200607,20060413,5,8,1,0.31,PRD0904471,CL00218,DEP00073,G00023,D00005,CUST0000325603,MM,YA,994100100347826,S,MM,Small Shop,Mixed,STORE02908,SS,N02
250544,200607,20060416,1,10,7,5.95,PRD0904660,CL00161,DEP00054,G00016,D00003,NaN,NaN,NaN,994100100107776,M,MM,Small Shop,Grocery,STORE02908,SS,N02
250545,200607,20060411,3,17,3,9.81,PRD0904933,CL00150,DEP00052,G00015,D00003,NaN,NaN,NaN,994100100038770,L,UM,Top Up,Grocery,STORE02908,SS,N02


## Descriptive Stats

In [137]:
df.describe(include='all')

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
count,250546.00,250546.00,250546.00,250546.00,250546.00,250546.00,250546,250546,250546,250546,250546,202211,202211,175047,250546.00,250546,250546,250546,250546,250546,250546,250546
unique,NaN,NaN,NaN,NaN,NaN,NaN,3809,242,89,31,9,17243,4,6,NaN,3,4,4,5,748,4,12
top,NaN,NaN,NaN,NaN,NaN,NaN,PRD0903052,CL00063,DEP00019,G00007,D00002,CUST0000701659,MM,OT,NaN,L,MM,Top Up,Fresh,STORE01423,LS,S02
freq,NaN,NaN,NaN,NaN,NaN,NaN,5968,11041,21616,56825,123764,101,91176,51914,NaN,178405,119486,113151,129265,1224,160648,26370
mean,200607.00,20060413.02,4.00,14.99,1.47,1.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994100100379997.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.00,2.01,2.01,3.72,1.41,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216601.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,200607.00,20060410.00,1.00,8.00,1.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994100100000007.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,200607.00,20060411.00,2.00,12.00,1.00,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994100100193201.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,200607.00,20060413.00,4.00,15.00,1.00,1.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994100100379527.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,200607.00,20060415.00,6.00,18.00,1.00,2.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,994100100568819.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Testing

## Data Types, Memory Usage, Nulls, Value Counts

In [138]:
#pandas documention re memory usage: base-2 representation; i.e. 1KB = 1024 bytes

In [139]:
def get_dataframe_info(df):
    """
Recreates column-wise info from df.info() as a DataFrame to allow for easier viewing from CSV
    """
    df_dtypes = pd.DataFrame(df.dtypes, columns=['Data Types'])

    df_memory_usage = df.memory_usage(index=False, deep=True).to_frame(name='Memory Use')
    df_memory_usage['Memory Use'] = df_memory_usage['Memory Use']/1024/1024

    df_percent_null = ((1 - df.count() / len(df)) * 100).to_frame(name='Percent Null')

    df_described = df.describe(include='all').T


    df_info = pd.concat([df_dtypes, df_memory_usage, df_percent_null, df_described], axis=1)

    # Reassign column names
    df_info = df_info.rename_axis('Column Names').round(2)

#    df_info = df_info.set_index('column_names')

    return df_info

In [140]:
df_info = get_dataframe_info(df)

In [141]:
df_info.style \
    .format(precision=2) \
    .set_sticky(axis='index') \
    .bar(color='#89d8e0')

,Data Types,Memory Use,Percent Null,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Column Names,,,,,,,,,,,,,,
SHOP_WEEK,int64,1.91,0.00,250546.00,nan,nan,nan,200607.00,0.00,200607.00,200607.00,200607.00,200607.00,200607.00
SHOP_DATE,int64,1.91,0.00,250546.00,nan,nan,nan,20060413.02,2.01,20060410.00,20060411.00,20060413.00,20060415.00,20060416.00
SHOP_WEEKDAY,int64,1.91,0.00,250546.00,nan,nan,nan,4.00,2.01,1.00,2.00,4.00,6.00,7.00
SHOP_HOUR,int64,1.91,0.00,250546.00,nan,nan,nan,14.99,3.72,8.00,12.00,15.00,18.00,21.00
QUANTITY,int64,1.91,0.00,250546.00,nan,nan,nan,1.47,1.41,1.00,1.00,1.00,1.00,275.00
SPEND,float64,1.91,0.00,250546.00,nan,nan,nan,1.93,4.00,0.01,0.76,1.23,2.11,1397.64
PROD_CODE,object,16.01,0.00,250546,3809,PRD0903052,5968,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_10,object,15.29,0.00,250546,242,CL00063,11041,nan,nan,nan,nan,nan,nan,nan
PROD_CODE_20,object,15.53,0.00,250546,89,DEP00019,21616,nan,nan,nan,nan,nan,nan,nan


### Save overview output in `.csv` and `.xlsx` format

In [142]:
#add a empty columns for note-taking in a spreadsheet
df_info.insert(loc=0,
               column='Notes',
               value = '')

df_info.insert(loc=1,
               column='Drop?',
               value = '')

df_info.insert(loc=2,
               column='Change Dtype?',
               value = '')

In [152]:
#write to excel file
df_info.to_excel(main_drive / 'notebooks' / 'eda' / f"{output_filename}_ov.xlsx",
                 sheet_name=f'Overview {file_path.stem}',
                 freeze_panes=(1,4)
                 )

## Correlation Table
For numeric columns only

In [153]:
df.corr(numeric_only=True)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,BASKET_ID
SHOP_WEEK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHOP_DATE,NaN,1.00,0.24,0.00,-0.00,-0.00,0.01
SHOP_WEEKDAY,NaN,0.24,1.00,-0.00,-0.00,0.00,-0.01
SHOP_HOUR,NaN,0.00,-0.00,1.00,-0.02,0.00,-0.06
QUANTITY,NaN,-0.00,-0.00,-0.02,1.00,0.35,-0.03
SPEND,NaN,-0.00,0.00,0.00,0.35,1.00,0.00
BASKET_ID,NaN,0.01,-0.01,-0.06,-0.03,0.00,1.00


## Duplicated Rows

In [154]:
df[df.duplicated(keep=False)]

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,PROD_CODE_40,CUST_CODE,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION


## Null Columns
List of columns with all NaN values

In [155]:
list(df.columns[df.isnull().all(axis=0)])

[]

# Columns

## List of Column Names

In [156]:
list(df.columns)

['SHOP_WEEK',
 'SHOP_DATE',
 'SHOP_WEEKDAY',
 'SHOP_HOUR',
 'QUANTITY',
 'SPEND',
 'PROD_CODE',
 'PROD_CODE_10',
 'PROD_CODE_20',
 'PROD_CODE_30',
 'PROD_CODE_40',
 'CUST_CODE',
 'CUST_PRICE_SENSITIVITY',
 'CUST_LIFESTAGE',
 'BASKET_ID',
 'BASKET_SIZE',
 'BASKET_PRICE_SENSITIVITY',
 'BASKET_TYPE',
 'BASKET_DOMINANT_MISSION',
 'STORE_CODE',
 'STORE_FORMAT',
 'STORE_REGION']

## Column Overview
This function loops over each column to produce the following info:
* name
* count of unique values
* datatype
* string to copy into a new cell, for displaying all value counts for a column

For value counts under the cutoff:
* a transposed display frame of all unique values

For value counts over the cutoff:
* transposed display frames of the top and bottom 10 values and counts


In [157]:
def column_overview(df, columns, cutoff=100):
    """
    Display column name, count of unique values, and an easy to read dataframe of individual unique values and their counts

    Parameters
    ----------
    df: dataframe
    columns: list
    cutoff: int, maximum unique value count to display
    Recommend string or object columns with unclear or low expected unique values.

    Returns
    -------
    Prints out one result for each column in the provided list.
    """
    for col in columns:
        print("\n")
        print("Column name: " + col)
        num_unique = str(df[col].nunique())
        print(f"Number of Unique Values: {num_unique}")
        col_datatype = str(df[col].dtype)
        print(f"Column Datatype: {col_datatype}")
        print("\n")

        if int(num_unique) < cutoff:
          print("use line below for vertical results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          display(pd.DataFrame(df[col].value_counts(dropna=False)).T)
          print(u'\u2500' * 80)
        else:
          print(f"more than {cutoff} results, showing Top 10 and Bottom 10")
          print("use line below for complete results")
          print(f"pd.DataFrame(df['{col}'].value_counts(dropna=False))")
          print("\n")
          print(f"Top 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).head(10)).T)
          print(f"Bottom 10 Unique Values of {col}")
          display(pd.DataFrame(df[col].value_counts(dropna=False).tail(10)).T)
          print(u'\u2500' * 80)


In [158]:
column_overview(df, df.columns, 200)



Column name: SHOP_WEEK
Number of Unique Values: 1
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEK'].value_counts(dropna=False))


,200607
SHOP_WEEK,250546


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_DATE
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_DATE'].value_counts(dropna=False))


,20060416,20060415,20060410,20060414,20060413,20060412,20060411
SHOP_DATE,36656,36183,35989,35922,35567,35183,35046


────────────────────────────────────────────────────────────────────────────────


Column name: SHOP_WEEKDAY
Number of Unique Values: 7
Column Datatype: int64


use line below for vertical results
pd.DataFrame(df['SHOP_WEEKDAY'].value_counts(dropna=False))


...

────────────────────────────────────────────────────────────────────────────────


Column name: STORE_FORMAT
Number of Unique Values: 4
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_FORMAT'].value_counts(dropna=False))


,LS,MS,XLS,SS
STORE_FORMAT,160648,51994,21781,16123


────────────────────────────────────────────────────────────────────────────────


Column name: STORE_REGION
Number of Unique Values: 12
Column Datatype: object


use line below for vertical results
pd.DataFrame(df['STORE_REGION'].value_counts(dropna=False))


,S02,N01,N03,W02,N02,W01,S01,S03,E03,E02,E01,W03
STORE_REGION,26370,24947,23024,22922,20669,20552,20278,20242,18591,18079,17876,16996


────────────────────────────────────────────────────────────────────────────────
